In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("CattleProdActionMatrixFactorization") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/06/01 16:04:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 16:04:47 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [3]:
from datetime import datetime

date_string = datetime.today().strftime('%Y-%m-%d')

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.linalg import DenseVector

# 从Hive中读取数据
df = spark.sql(f'''
SELECT
    user_id, 
    item_id, 
    val 
FROM 
    cattle_prod_user_action_matrix 
where 
    date = '{date_string}'
''')

# 初始化ALS模型
als = ALS(rank=15, maxIter=10,userCol="user_id", itemCol="item_id", ratingCol="val")

# 训练模型
model = als.fit(df)

# 获取用户和物品的因子
userFactors = model.userFactors
itemFactors = model.itemFactors

# 将因子分解为单独的列
def extract(row):
    return (row.id,) + tuple(float(x) for x in row.features)

userFactors = userFactors.rdd.map(extract).toDF(["user_id"] + ["fac_" + str(i) for i in range(model.rank)])
itemFactors = itemFactors.rdd.map(extract).toDF(["item_id"] + ["fac_" + str(i) for i in range(model.rank)])

# 将因子数据保存回Hive
userFactors.write.mode("overwrite").saveAsTable("cattle_prod_user_fact_matrix")
itemFactors.write.mode("overwrite").saveAsTable("cattle_prod_item_fact_matrix")


23/06/01 16:09:49 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/06/01 16:10:00 ERROR util.Instrumentation: java.lang.IllegalArgumentException: requirement failed: No ratings available from MapPartitionsRDD[6] at map at ALS.scala:699
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.recommendation.ALS$.train(ALS.scala:947)
	at org.apache.spark.ml.recommendation.ALS.$anonfun$fit$1(ALS.scala:709)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.recommendation.ALS.fit(ALS.scala:691)
	at org.apache.spark.ml.recommendation.ALS.fit(ALS.scala:593)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAcc

IllegalArgumentException: requirement failed: No ratings available from MapPartitionsRDD[6] at map at ALS.scala:699

In [ ]:
spark.stop()